# Laboratorio 3-04: Método del descenso más rápido

Samantha Sánchez Tinoco

In [1]:
import numpy as np

In [3]:
# Método Gauss Jordan para resolver el sistema de alfas
def metodo_gauss(a, b):
    n = len(b)
    a = [[float(num) for num in fila] for fila in a]
    b = [float(num) for num in b]

    # Reducción de Gauss
    for i in range(n):
        if abs(a[i][i]) < 1e-12:
            print("No hay solución única.")
            return None
        
        # Normalización de la fila pivote
        for j in range(i + 1, n):
            factor = a[j][i] / a[i][i]
            for k in range(i, n):
                a[j][k] -= factor * a[i][k]
            b[j] -= factor * b[i]

    # Sustitución hacia atrás para resultados
    x = [0] * n
    for i in range(n - 1, -1, -1):
        suma = sum(a[i][j] * x[j] for j in range(i + 1, n))
        x[i] = (b[i] - suma) / a[i][i]

    return x

In [4]:
# Función h(alfa) para evaluar el sistema de ecuaciones
def h_alfa(x_k, alfa, Ucm):
    x_temp = x_k - alfa * Ucm
    x1, x2, x3 = x_temp
    f1 = 3 * x1 - 81 * (x2 * x3) - 1 / 2
    f2 = x1 ** 2 - 81 * (x2 + 0.1) ** 2 + np.sin(x3) + 1.06
    f3 = np.exp(-x1 * x2) + 20 * x3 + (10 * np.pi - 3) / 3
    return f1**2 + f2**2 + f3**2

In [5]:
# Valores del X2 calculados en la tarea 3-03
x_k = np.array([0.01125, -0.01606, -0.52388], dtype=float)
tol = 1e-4
iter_count = 0

In [8]:
while True:
    iter_count += 1

    # Sistema de ecuaciones y F(x)
    x1, x2, x3 = x_k
    f1 = 3 * x1 - np.cos(x2 * x3) - 1 / 2
    f2 = x1 ** 2 - 81 * (x2 + 0.1) ** 2 + np.sin(x3) + 1.06
    f3 = np.exp(-x1 * x2) + 20 * x3 - (10 * np.pi - 3) / 3
    Fx = np.array([f1, f2, f3])

    # Jacobiano J(F)
    JF = np.array([
    [3, x3 * np.sin(x2 * x3), x2 * np.sin(x2 * x3)],
    [2 * x1, -162 * (x2 + 0.1), np.cos(x3)],
    [-x2 * np.exp(-x1 * x2), -x1 * np.exp(-x1 * x2), 20]
    ])

    # Gradiente de g(x) = 2J(F)F y su norma
    grad_g = 2 * JF.T @ Fx
    norm_grad_g = np.linalg.norm(grad_g)

    # Ucm
    Ucm = grad_g / norm_grad_g

    # Valores iniciales de alfas
    alfa3 = 0
    incremento = 1
    while True:
        alfa1 = alfa3 - incremento
        alfa2 = (alfa1 + alfa3) / 2

        # Evaluación de h(alfa) para cada alfa
        h_alfa1 = h_alfa(x_k, alfa1, Ucm)
        h_alfa2 = h_alfa(x_k, alfa2, Ucm)
        h_alfa3 = h_alfa(x_k, alfa3, Ucm)

        # Comprobación de condición para h(alfa)
        if h_alfa3 < h_alfa1:
            break
        alfa3 += incremento

    # Sistema de ecuaciones de alfas
    A = [
        [alfa1 ** 2, alfa1, 1],
        [alfa2 ** 2, alfa2, 1],
        [alfa3 ** 2, alfa3, 1]
    ]
    B = [h_alfa1, h_alfa2, h_alfa3]

    # Resolver a, b y c usando el método de Gauss
    coeficientes = metodo_gauss(A, B)
    if coeficientes is None:
        print("No se pudo encontrar una solución.")
        break

    a, b, c = coeficientes

    # Cálculo del alfa óptimo
    alfa = -b / (2 * a)

    # Actualización de x_k+1
    x_k1 = x_k - alfa * Ucm

    # Comprobar la tolerancia ||x(k+1) - x(k)|| < tol
    if np.linalg.norm(x_k1 - x_k) < tol:
        print(f"Convergencia en {iter_count} iteraciones.")
        print("Solución:", x_k1)
        break

    # Actualización para la siguiente iteración
    x_k = x_k1

Convergencia en 4 iteraciones.
Solución: [ 0.01130507 -0.0160643  -0.519442  ]
